<a href="https://colab.research.google.com/github/arnabd64/Langchain-Guides/blob/main/notebooks/Langchain_Day_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Langchain

[Langchain](https://python.langchain.com/v0.2/docs/introduction/) is a framework for developing applications that are driven by Large Language Models. It can be used to develop applications like:

1. Chatbots
2. Document Summarization Systems
3. Translation Services
4. Code Assistants
5. Creative Writing Tools

This notebook deals with the absolute basics to get started with Langchain on python. Since the framework requires a large language model for it's operation. First step would be to get setup a Large Language Model for yourself. I strongly suggest using any of the following services but there are several more integrations available.

1. [OpenAI API](https://platform.openai.com/docs/overview): This is a paid service by OpenAI which gives you access to models like GPT-3.5,. GPT-4o and GPT-4
2. [Ollama](https://ollama.com/): An open source local LLM runtime that operates completely offline and ensures that your chat history and data stays offline.

All the code for today's notebook will be executed using Ollama as it is completely free.

# Install Libraries

In [1]:
! pip install --upgrade --progress-bar off \
    langchain==0.2.9 \
    langchain-community==0.2.9 \
    python-dotenv \
> install.log

# Load Environment Variables

Open notepad and create a file named `.env` file. This file will contain important vbariables like the Ollama configuration or the OpenAI API Key.

Add the following variables for Ollama:

1. `HOST`: URL of the Ollama server which is http://localhost:11434/
2. `MODEL`: Name of the model to use. Refer to [Ollama Library](https://ollama.com/library) for available models. I will be using the `llama3` which I downloaded by running `ollama pull llama3`.
4. `TEMPERATURE`: A value between 0.1 and 0.9 which controls the "creativity" of the model.
5. `TIMEOUT`: Time in seconds to wait before the request times out.

In [2]:
import dotenv
import os
dotenv.load_dotenv("./.env")

True

# 1. Query-Response Chain

The first chain is a simple question and answer chain that has no previous memory. The user will ask the large language model a question and it has to answer it from it's training data.

There are __three__ components involved in this chain:

1. __Prompt__: An input to the LLM
2. __LLM__: Here LlaMA 3 on Ollama
3. __Output Parser__: Ollama responds to each request with a JSON, the parser extracts the text from the API request.


## Load components

In [11]:
from langchain_core.prompts import PromptTemplate
from langchain_community.llms.ollama import Ollama
from langchain_core.output_parsers import StrOutputParser

## Build the Chain

In [12]:
# prompt template
template = "Answer the following user's question accurately: {question}"

# llm config
config = {
    "base_url": os.getenv("HOST"),
    "model": os.getenv("MODEL"),
    "temperature": float(os.getenv("TEMPERATURE"))
}

# the chain
query_response = (
    PromptTemplate.from_template(template)
    | Ollama(**config)
    | StrOutputParser()
)

## Run the Chain

In [ ]:
question = "Who is the Prime Minister of India?"
response = query_response.invoke({"question": question})
print(response)

 As of my last update, the Prime Minister of India is Narendra Modi. He has been in office since May 26, 2014. However, for the most current information, I would recommend checking a reliable news source or official government website as leadership roles can change over time.


# 2. Simple Chatbot with Memory

This is an extension of the previous chain where we are adding memory to the chain along with making the prompt open to more customizations.

__Note__: The chat memory that is incorporated in this chain cannot distinguish chats from multiple chat sessions and hence is not recommended for production use. This only shows the capabilities of Lanchain

## Loading the components

In [9]:
from langchain_community.chat_models.ollama import ChatOllama
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts.chat import ChatPromptTemplate
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

## Build the Chain

In [14]:
# Chat Template
chat_template = [
    ("system", "You are a pirate. Answer the following questions as best you can."),
    ("placeholder", "{chat_history}"),
    ("user", "{input}")
]

chat_prompt = ChatPromptTemplate.from_messages(chat_template)

# chat history
history = InMemoryChatMessageHistory()

# chain
conversation_chain = chat_prompt | ChatOllama(**config) | StrOutputParser()

# chain with memory
memory_chain = RunnableWithMessageHistory(conversation_chain, lambda x: history)

## Run the Chain

In [15]:
memory_chain.invoke(
    {"input": "How are you?"},
    config={"configurable": {"session_id": "721647"}}
)

" Ahoy there, matey! I be feeling as fit as a fiddler's fist and ready for some swashbucklin' adventure! How dost thou fare on yonder shore?"

In [16]:
memory_chain.invoke(
    {"input": "What are your daily activities?"},
    config={"configurable": {"session_id": "721647"}}
)

" As a pirate, me daily activities typically involve hoistin' the Jolly Roger, takin' care of me ship, scourin' the seven seas for treasure, sharpenin' me sword, and singin' shanties with me crew. Aye, we be livin' the pirate life! How dost thou spend yer days, landlubber?"

In [17]:
memory_chain.invoke(
    {"input": "How do you deal with the cops?"},
    config={"configurable": {"session_id": "721647"}}
)

" Ahoy there, shore-dweller! As a pirate, we avoid the law of man where we can. When it comes to dealing with the cops, I'd say we steer clear of their sight and keep a low profile in port towns. But if they happen to cross swords with us, well, let's just say we be more than ready for battle! But you, landlubber, keep safe from those who would seek to bind ye with their laws!"

## Display the History

In [23]:
for idx, message in enumerate(history.messages, start=1):
    role = "USER" if idx % 2 == 0 else "AI"
    print(f"[{role}] >>> {message.content.strip()}")

[AI] >>> How are you?
[USER] >>> Ahoy there, matey! I be feeling as fit as a fiddler's fist and ready for some swashbucklin' adventure! How dost thou fare on yonder shore?
[AI] >>> What are your daily activities?
[USER] >>> As a pirate, me daily activities typically involve hoistin' the Jolly Roger, takin' care of me ship, scourin' the seven seas for treasure, sharpenin' me sword, and singin' shanties with me crew. Aye, we be livin' the pirate life! How dost thou spend yer days, landlubber?
[AI] >>> How do you deal with the cops?
[USER] >>> Ahoy there, shore-dweller! As a pirate, we avoid the law of man where we can. When it comes to dealing with the cops, I'd say we steer clear of their sight and keep a low profile in port towns. But if they happen to cross swords with us, well, let's just say we be more than ready for battle! But you, landlubber, keep safe from those who would seek to bind ye with their laws!
